<a href="https://colab.research.google.com/github/KevinFallon/fastai/blob/main/FastAITutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from fastai.tabular.all import *

In [2]:
path = untar_data(URLs.ADULT_SAMPLE)
path.ls()

(#3) [Path('/root/.fastai/data/adult_sample/adult.csv'),Path('/root/.fastai/data/adult_sample/export.pkl'),Path('/root/.fastai/data/adult_sample/models')]

In [3]:
df = pd.read_csv(path/'adult.csv')
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,49,Private,101320,Assoc-acdm,12.0,Married-civ-spouse,NaN,Wife,White,Female,0,1902,40,United-States,>=50k
1,44,Private,236746,Masters,14.0,Divorced,Exec-managerial,Not-in-family,White,Male,10520,0,45,United-States,>=50k
2,38,Private,96185,HS-grad,NaN,Divorced,NaN,Unmarried,Black,Female,0,0,32,United-States,<50k
3,38,Self-emp-inc,112847,Prof-school,15.0,Married-civ-spouse,Prof-specialty,Husband,Asian-Pac-Islander,Male,0,0,40,United-States,>=50k
4,42,Self-emp-not-inc,82297,7th-8th,NaN,Married-civ-spouse,Other-service,Wife,Black,Female,0,0,50,United-States,<50k


In [4]:
dls = TabularDataLoaders.from_csv(path/'adult.csv', path=path, y_names='salary',
                                  cat_names = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race'],
                                  cont_names = ['age', 'fnlwgt', 'education-num'],
                                  procs = [Categorify, FillMissing, Normalize]
                                  )

In [5]:
splits = RandomSplitter(valid_pct=0.2)(range_of(df))
to = TabularPandas(df, procs=[Categorify, FillMissing, Normalize],
                   cat_names = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race'],
                   cont_names = ['age', 'fnlwgt', 'education-num'],
                   y_names='salary',
                   splits=splits,
                   )
to.xs.iloc[:2]

,workclass,education,marital-status,occupation,relationship,race,education-num_na,age,fnlwgt,education-num
32170,5,3,3,15,1,5,1,-0.701781,-0.539747,-0.820465
15090,5,12,5,7,2,5,1,-0.481747,0.161253,-0.426607


In [6]:
dls = to.dataloaders(bs=64)
dls.show_batch()

,workclass,education,marital-status,occupation,relationship,race,education-num_na,age,fnlwgt,education-num,salary
0,Private,Some-college,Never-married,Prof-specialty,Not-in-family,Other,False,26.000000,184025.999982,10.0,<50k
1,Private,HS-grad,Separated,Other-service,Unmarried,White,False,25.000000,108300.998987,9.0,<50k
2,Private,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,False,44.000000,145441.001368,13.0,>=50k
3,Private,5th-6th,Never-married,Machine-op-inspct,Not-in-family,White,False,52.000000,155432.999678,3.0,<50k
4,Private,HS-grad,Never-married,Other-service,Own-child,White,False,31.000000,173495.000121,9.0,<50k
5,Private,HS-grad,Married-civ-spouse,Exec-managerial,Husband,White,False,58.000000,114494.998504,9.0,<50k
6,Private,HS-grad,Married-civ-spouse,Transport-moving,Own-child,White,False,24.000000,211584.999594,9.0,<50k
7,Local-gov,HS-grad,Married-civ-spouse,Protective-serv,Husband,Black,False,19.999999,312427.002761,9.0,<50k
8,Private,Doctorate,Married-civ-spouse,Prof-specialty,Husband,White,False,50.000000,177896.000317,16.0,>=50k
9,Local-gov,Some-college,Married-civ-spouse,Protective-serv,Husband,White,False,54.000001,204325.000642,10.0,<50k


In [7]:
learn = tabular_learner(dls, metrics=accuracy)

learn.fit_one_cycle(1)

epoch,train_loss,valid_loss,accuracy,time
0,0.369952,0.359685,0.834152,00:10


In [8]:
learn.show_results()

,workclass,education,marital-status,occupation,relationship,race,education-num_na,age,fnlwgt,education-num,salary,salary_pred
0,5.0,16.0,3.0,4.0,1.0,5.0,1.0,-0.555092,0.940373,-0.032750,0.0,0.0
1,5.0,12.0,7.0,11.0,2.0,5.0,1.0,1.791935,-0.524973,-0.426607,0.0,0.0
2,6.0,12.0,3.0,2.0,6.0,5.0,1.0,2.378692,-0.533272,-0.426607,1.0,1.0
3,5.0,12.0,5.0,4.0,4.0,5.0,1.0,-1.215193,0.614481,-0.426607,0.0,0.0
4,7.0,6.0,3.0,5.0,1.0,5.0,1.0,2.158658,0.481418,-2.395895,0.0,0.0
5,5.0,8.0,5.0,2.0,2.0,5.0,1.0,-0.481747,0.776336,0.754966,0.0,0.0
6,3.0,1.0,5.0,9.0,4.0,5.0,1.0,-1.581916,-1.421807,-1.608180,0.0,0.0
7,5.0,10.0,1.0,5.0,2.0,5.0,1.0,0.325043,-0.148069,1.148823,1.0,1.0
8,3.0,16.0,1.0,15.0,5.0,5.0,1.0,0.618421,-0.230802,-0.032750,0.0,0.0


In [9]:
row, clas, probs = learn.predict(df.iloc[0])
row.show()
clas, probs

,workclass,education,marital-status,occupation,relationship,race,education-num_na,age,fnlwgt,education-num,salary
0,Private,Assoc-acdm,Married-civ-spouse,#na#,Wife,White,False,49.0,101319.997801,12.0,>=50k


(tensor(1), tensor([0.3215, 0.6785]))

In [10]:
test_df = df.copy()
test_df.drop(['salary'], axis=1, inplace=True)
dl = learn.dls.test_dl(test_df)
learn.get_preds(dl=dl)

(tensor([[0.3215, 0.6785],
         [0.3014, 0.6986],
         [0.9158, 0.0842],
         ...,
         [0.5402, 0.4598],
         [0.7040, 0.2960],
         [0.7670, 0.2330]]), None)